In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from numpy import linalg
from scipy.optimize import linprog, minimize_scalar
from scipy.stats import norm, chi2

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [3]:
import sys
import os
sys.path.append('../utils')
from algorithms_1 import  Bisection, GD, Ellipsoid, GD_m, AGD_m, AGD, LIBRARY, LIBRARY_m
from scipy.optimize import minimize
import time
import math

In [4]:
from sklearn import datasets
my_data = datasets.load_svmlight_file("cod-rna.txt")

In [5]:
data0 = my_data[0]

In [6]:
dim = 8         #m
n_p = 4000      #m+
n_n = 4000
n = n_n + n_p   #m-
p = 3           #p
data_p = data0[:n_p].todense()
data_n = data0[-n_p:].todense()
data = np.vstack((data_p,data_n))

sigma = 10**np.linspace(-1., 2., p)
a = np.ones(n)
a[0:n_p] = np.ones(n_p) / n_p
a[n_p:n] = -np.ones(n_n) / n_n

G = np.zeros((p, n, n))
G_tilde = np.zeros((p, n, n))
P =  - 1. / n * np.outer(np.ones(n),np.ones(n)) + np.eye(n) 
r = np.zeros(p)

t0 = time.time()
for i in range(p):
    for j in range(n):
        for k in range(n):
            G[i,j,k] = np.exp(- np.linalg.norm(data[j] - data[k])**2 / sigma[i]**2)


for i in range(p):
    G_tilde[i] = P.dot(G[i].dot(P))
    r[i] = np.trace(G_tilde[i])
print(time.time() -  t0)

2361.770831346512


In [7]:
import sys
import numpy as np
import cvxpy as cp
import time
import mosek
from collections import OrderedDict

In [8]:
A = np.zeros((p , n, n ))
b =  np.zeros((p, n ))
c =  np.zeros(p)

a_obj = a
lambda_obj = 1e-4
t_obj = 5e-8
params = []

for i in range(p):
    A[i] = 1./ r[i] * G_tilde[i]
    params.append((A[i]))
    print(np.linalg.norm(A[i]))
    
params.append((a_obj,lambda_obj,t_obj))

0.014788997437451074
0.06093207656189058
0.43435598439403716


In [9]:
# Objective function

def f_objective4(beta, x_p, params):
    nn = beta.size
    a_obj, lambda_obj, t_obj = params[-1]
    return 0.25 * beta.dot(beta) -   beta.dot(a_obj) +  0.25 / lambda_obj * t_obj 

def f_grad4(beta, x_p, params):
    nn = beta.size
    a_obj, lambda_obj, t_obj = params[-1]
    f_grad_vector = 0.5 * beta -  a_obj
    return f_grad_vector

# Constraints 

def h_constraint4(beta, params):
    m = len(params) - 1
    h_con = np.zeros(m)
    a_obj, lambda_obj, t_obj = params[-1]
#     print("beta",beta,"m",m)
    for i in range(m):
        A = params[i]
        h_con[i] =  beta.dot(A.dot(beta)) -  t_obj
    return h_con

def h_grad4(beta, params):
    nn = beta.size
    m = len(params) - 1
    dh = np.zeros((m, nn))
    for i in range(m):
        A = params[i]
        #print(A,b,c, dh[i])
        dh[i] = 2 * A.dot(beta)
    #print(1)
    return dh

In [54]:
# Fast Projections, eps = 10-8
x_0 = np.zeros(n)
x_p = np.zeros(n)
l_up = 500.
epsilon = 1e-8
mu = 0.5
t_obj = 5e-8
L =  0.5 
learning_rate = 1 / 2 / L
K_max = 2500
T = 100
t0 = time.time()
x_g, l_g, v_g = Ellipsoid(l_up, epsilon, x_0, x_p, 
                             L, T, AGD_m, 
                             h_grad4, h_constraint4, 
                             learning_rate, params, 
                             f_objective4, f_grad4, 
                          K_max = K_max, mu = mu, 
                          G = 0.001)
time_ellipsoid = time.time() - t0
print(l_g, v_g, time_ellipsoid)

crit_reached
57
[197.12718423   4.37123757 148.30888346] -2.2213487328810266e-05 1216.944081544876


In [52]:
# Fast Projections, eps = 10-7
x_0 = np.zeros(n)
x_p = np.zeros(n)
l_up = 500.
epsilon = 1e-7
mu = 0.5
t_obj = 5e-8
L =  0.5
learning_rate = 1 / 2 / L
K_max = 2500
T = 50
t0 = time.time()
x_g, l_g, v_g = Ellipsoid(l_up, epsilon, x_0, x_p, 
                             L, T, AGD_m, 
                             h_grad4, h_constraint4, 
                             learning_rate, params, 
                             f_objective4, f_grad4, 
                             K_max = K_max, mu = mu, 
                             G = 0.001)
time_ellipsoid = time.time() - t0
print(l_g, v_g, time_ellipsoid)

crit_reached
32
[208.30688522   5.07345614 142.98871411] -2.2535031338787265e-05 519.7166244983673


In [10]:
# Fast Projections, eps = 10-6
x_0 = np.zeros(n)
x_p = np.zeros(n)
l_up = 500.
epsilon = 1e-6
mu = 0.5
t_obj = 5e-8
L =  0.5 
learning_rate = 1 / 2 / L
K_max = 2500
T = 50
t0 = time.time()
x_g, l_g, v_g = Ellipsoid(l_up, epsilon, x_0, x_p, 
                             L, T, AGD_m, 
                             h_grad4, h_constraint4, 
                             learning_rate, params, 
                             f_objective4, f_grad4, K_max = K_max, mu = mu, G = 0.001)
time_ellipsoid = time.time() - t0
print(l_g, v_g, time_ellipsoid)

48
[499.58813849  44.60433299 165.00396218] -0.00010982592095649772 338.0227544307709


## MOSEK

In [11]:
def Solve_Mosek_3(a_obj, lambda_obj, A, tol):
    beta = cp.Variable((n))
    t = 5e-8
    a_cp = cp.Constant(a_obj)
    

    constraints = [
         cp.quad_form(beta, A[0]) -  t <= 0,
         cp.quad_form(beta, A[1]) -  t <= 0,
         cp.quad_form(beta, A[2]) -  t <= 0
    ]
    
    objective = cp.Minimize(0.25 * cp.norm(beta) ** 2 - beta @ a_cp + 0.25 * t / lambda_obj)

    prob = cp.Problem(objective, constraints)
    prob.solve(solver = cp.MOSEK, mosek_params = {mosek.dparam.intpnt_co_tol_rel_gap: tol,
                              mosek.dparam.intpnt_co_tol_dfeas: tol,
                              mosek.dparam.intpnt_co_tol_infeas: tol * 1.0e-2,
                              mosek.dparam.intpnt_co_tol_mu_red: tol,
                              mosek.dparam.intpnt_co_tol_pfeas: tol,
                              })

    return prob

In [40]:
# MOSEK (small original problem with fixed t, n=5500,m=3, eps = 10-8)
tol = 1.0e-8
t0 = time.time()
prob = Solve_Mosek_3(a_obj, lambda_obj, A, tol)
time_mosek = time.time() - t0
primal = prob.solution.primal_vars
prob_d = prob.solution.dual_vars
val = prob.value
print( " dual: ", prob_d, "value: ", val, "time:", time_mosek)

 dual:  {260: array([195.64394114]), 267: array([8.76804549]), 274: array([145.031198])} value:  -2.2260651924760425e-05 time: 1302.0883221626282


In [48]:
# MOSEK (small original problem with fixed t, n=5500,m=3, eps = 10-7)
tol = 1.0e-7
t0 = time.time()
prob = Solve_Mosek_3(a_obj, lambda_obj, A, tol)
time_mosek = time.time() - t0
primal = prob.solution.primal_vars
prob_d = prob.solution.dual_vars
val = prob.value
print( " dual: ", prob_d, "value: ", val, "time:", time_mosek)

 dual:  {895: array([227.23238466]), 902: array([32.20507064]), 909: array([89.63268749])} value:  -2.4932902869652845e-05 time: 1062.1518244743347


In [12]:
# MOSEK (small original problem with fixed t, n=5500,m=3, eps = 10-6)
tol = 1.0e-6
t0 = time.time()
prob = Solve_Mosek_3(a_obj, lambda_obj, A, tol)
time_mosek = time.time() - t0
primal = prob.solution.primal_vars
prob_d = prob.solution.dual_vars
val = prob.value
print( " dual: ", prob_d, "value: ", val, "time:", time_mosek)

Error: rescode.err_license_expired(1001): The license has expired.